# k=50

In [3]:
#-*- coding: utf-8 -*-
#K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

inputfile = '201404.xlsx' #待聚类的数据文件
k = 50                       #需要进行的聚类类别数

#读取数据并进行聚类分析
data = pd.read_excel(inputfile) #读取数据

#调用k-means算法，进行聚类分析
kmodel = KMeans(n_clusters = k, n_jobs = 2) #n_jobs是并行数，一般等于CPU数较好
kmodel.fit(data) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=50, n_init=10,
    n_jobs=2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [4]:
data['label']=kmodel.labels_

In [6]:
data.head()

,X2,Y2,label
0,121.59980,38.90740,7
1,121.55690,38.88571,1
2,121.57105,38.89943,38
3,121.59801,38.90098,25
4,121.56325,38.89774,38


In [7]:
data.to_excel('2014_cluster_50.xlsx',index=None) #设置index=None，存为的excel文件就不会多索引列

创建类中心点文件

In [8]:
center=pd.DataFrame(kmodel.cluster_centers_,columns=['X2','Y2'])
center.to_excel('2014center_50.xlsx',index=None)

# NSGAII generate bus stops

In [11]:
# -*- coding: UTF-8 -*-
import random
import xlrd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import math

random.seed(64)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def getItems():

    fname = "2014center_50.xlsx"
    bk = xlrd.open_workbook(fname)

    try:
        sh = bk.sheet_by_name("Sheet1")
    except:
        print "no sheet in %s named Sheet1" % fname
    #获取行数
    nrows = sh.nrows
    #获取列数
    ncols = sh.ncols
    # print "nrows %d, ncols %d" % (nrows,ncols)
    #获取第一行第一列数据
    cell_value = sh.cell_value(1,1)
    row_list = []
    #获取各行数据
    items = {}
    for i in range(0,nrows):
        row_data = sh.row_values(i)
        # row_list.append(row_data)
        items[i]=row_data
    return items

items = {}
items = getItems()
toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 50)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ( 'fitness') function to be maximized

def evalStop(individual):
    cost = 0.0
    distance = 0.0
    print individual
    selected = []
    remainded = []
    temp_dist = [] #临时存储计算距离，用来比较选择剩下点到已选点最短路径
    dist = []
    cost = sum(individual)
    for index,item in enumerate(individual):
        if item == 1:
            selected.append(index)
        else:
            remainded.append(index)
    # print selected
    # print remainded
    for r in selected:
        for s in remainded:
            temp =math.sqrt((items[s][0]-items[r][0])**2+(items[s][1]-items[r][1])**2)
            temp_dist.append(temp)
        #print temp_dist
        if len(temp_dist) == 0:
            dist.append(0)
        else:
            dist.append(min(temp_dist))

    distance = sum(dist)
    return cost, distance

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalStop)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selNSGA2)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=50)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print evalStop(best_ind)
if __name__ == "__main__":
    main()


Start of evolution
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,

# k=100

In [33]:
#-*- coding: utf-8 -*-
#K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

inputfile = '201404.xlsx' #待聚类的数据文件
k_100 = 100                      #需要进行的聚类类别数

#读取数据并进行聚类分析
data_100 = pd.read_excel(inputfile) #读取数据

#调用k-means算法，进行聚类分析
kmodel_100 = KMeans(n_clusters = k_100, n_jobs = 2) #n_jobs是并行数，一般等于CPU数较好
kmodel_100.fit(data_100) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=100, n_init=10,
    n_jobs=2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [34]:
data_100['label']=kmodel_100.labels_

In [35]:
data_100.to_excel('2014_cluster_100.xlsx',index=None) #设置index=None，存为的excel文件就不会多索引列

In [36]:
center=pd.DataFrame(kmodel_100.cluster_centers_)
center.to_excel('2014center_100.xlsx',index=None)

In [37]:
# -*- coding: UTF-8 -*-
import random
import xlrd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import math

random.seed(64)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def getItems():

    fname = "2014center_100.xlsx"
    bk = xlrd.open_workbook(fname)

    try:
        sh = bk.sheet_by_name("Sheet1")
    except:
        print "no sheet in %s named Sheet1" % fname
    #获取行数
    nrows = sh.nrows
    #获取列数
    ncols = sh.ncols
    # print "nrows %d, ncols %d" % (nrows,ncols)
    #获取第一行第一列数据
    cell_value = sh.cell_value(1,1)
    row_list = []
    #获取各行数据
    items = {}
    for i in range(0,nrows):
        row_data = sh.row_values(i)
        # row_list.append(row_data)
        items[i]=row_data
    return items

items = {}
items = getItems()
toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 50)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ( 'fitness') function to be maximized

def evalStop(individual):
    cost = 0.0
    distance = 0.0
    print individual
    selected = []
    remainded = []
    temp_dist = [] #临时存储计算距离，用来比较选择剩下点到已选点最短路径
    dist = []
    cost = sum(individual)
    for index,item in enumerate(individual):
        if item == 1:
            selected.append(index)
        else:
            remainded.append(index)
    # print selected
    # print remainded
    for r in selected:
        for s in remainded:
            temp =math.sqrt((items[s][0]-items[r][0])**2+(items[s][1]-items[r][1])**2)
            temp_dist.append(temp)
        #print temp_dist
        if len(temp_dist) == 0:
            dist.append(0)
        else:
            dist.append(min(temp_dist))

    distance = sum(dist)
    return cost, distance

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalStop)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selNSGA2)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=50)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print evalStop(best_ind)
if __name__ == "__main__":
    main()


Start of evolution
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,

# k=150

In [38]:
#-*- coding: utf-8 -*-
#K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

inputfile = '201404.xlsx' #待聚类的数据文件
k_150 = 150                       #需要进行的聚类类别数

#读取数据并进行聚类分析
data_150 = pd.read_excel(inputfile) #读取数据

#调用k-means算法，进行聚类分析
kmodel_150 = KMeans(n_clusters = k_150, n_jobs = 2) #n_jobs是并行数，一般等于CPU数较好
kmodel_150.fit(data_150) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=150, n_init=10,
    n_jobs=2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [39]:
data_150['label']=kmodel_150.labels_
data_150.to_excel('2014_cluster_150.xlsx',index=None) #设置index=None，存为的excel文件就不会多索引列

In [40]:
center=pd.DataFrame(kmodel_150.cluster_centers_)
center.to_excel('2014center_150.xlsx',index=None)

In [41]:
# -*- coding: UTF-8 -*-
import random
import xlrd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import math

random.seed(64)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def getItems():

    fname = "2014center_150.xlsx"
    bk = xlrd.open_workbook(fname)

    try:
        sh = bk.sheet_by_name("Sheet1")
    except:
        print "no sheet in %s named Sheet1" % fname
    #获取行数
    nrows = sh.nrows
    #获取列数
    ncols = sh.ncols
    # print "nrows %d, ncols %d" % (nrows,ncols)
    #获取第一行第一列数据
    cell_value = sh.cell_value(1,1)
    row_list = []
    #获取各行数据
    items = {}
    for i in range(0,nrows):
        row_data = sh.row_values(i)
        # row_list.append(row_data)
        items[i]=row_data
    return items

items = {}
items = getItems()
toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 50)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ( 'fitness') function to be maximized

def evalStop(individual):
    cost = 0.0
    distance = 0.0
    print individual
    selected = []
    remainded = []
    temp_dist = [] #临时存储计算距离，用来比较选择剩下点到已选点最短路径
    dist = []
    cost = sum(individual)
    for index,item in enumerate(individual):
        if item == 1:
            selected.append(index)
        else:
            remainded.append(index)
    # print selected
    # print remainded
    for r in selected:
        for s in remainded:
            temp =math.sqrt((items[s][0]-items[r][0])**2+(items[s][1]-items[r][1])**2)
            temp_dist.append(temp)
        #print temp_dist
        if len(temp_dist) == 0:
            dist.append(0)
        else:
            dist.append(min(temp_dist))

    distance = sum(dist)
    return cost, distance

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalStop)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selNSGA2)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=50)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print evalStop(best_ind)
if __name__ == "__main__":
    main()


Start of evolution
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,

# k=200

In [42]:
#-*- coding: utf-8 -*-
#K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

inputfile = '201404.xlsx' #待聚类的数据文件
k_200 = 200                      #需要进行的聚类类别数

#读取数据并进行聚类分析
data_200 = pd.read_excel(inputfile) #读取数据

#调用k-means算法，进行聚类分析
kmodel_200 = KMeans(n_clusters = k_200, n_jobs = 2) #n_jobs是并行数，一般等于CPU数较好
kmodel_200.fit(data_200) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=200, n_init=10,
    n_jobs=2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [44]:
data_200['label']=kmodel_200.labels_
data_200.to_excel('2014_cluster_200.xlsx',index=None) #设置index=None，存为的excel文件就不会多索引列

In [45]:
center=pd.DataFrame(kmodel_200.cluster_centers_)
center.to_excel('2014center_200.xlsx',index=None)

In [46]:
# -*- coding: UTF-8 -*-
import random
import xlrd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import math

random.seed(64)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def getItems():

    fname = "2014center_200.xlsx"
    bk = xlrd.open_workbook(fname)

    try:
        sh = bk.sheet_by_name("Sheet1")
    except:
        print "no sheet in %s named Sheet1" % fname
    #获取行数
    nrows = sh.nrows
    #获取列数
    ncols = sh.ncols
    # print "nrows %d, ncols %d" % (nrows,ncols)
    #获取第一行第一列数据
    cell_value = sh.cell_value(1,1)
    row_list = []
    #获取各行数据
    items = {}
    for i in range(0,nrows):
        row_data = sh.row_values(i)
        # row_list.append(row_data)
        items[i]=row_data
    return items

items = {}
items = getItems()
toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 50)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ( 'fitness') function to be maximized

def evalStop(individual):
    cost = 0.0
    distance = 0.0
    print individual
    selected = []
    remainded = []
    temp_dist = [] #临时存储计算距离，用来比较选择剩下点到已选点最短路径
    dist = []
    cost = sum(individual)
    for index,item in enumerate(individual):
        if item == 1:
            selected.append(index)
        else:
            remainded.append(index)
    # print selected
    # print remainded
    for r in selected:
        for s in remainded:
            temp =math.sqrt((items[s][0]-items[r][0])**2+(items[s][1]-items[r][1])**2)
            temp_dist.append(temp)
        #print temp_dist
        if len(temp_dist) == 0:
            dist.append(0)
        else:
            dist.append(min(temp_dist))

    distance = sum(dist)
    return cost, distance

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalStop)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selNSGA2)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=50)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print evalStop(best_ind)
if __name__ == "__main__":
    main()


Start of evolution
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,

# k=250

In [47]:
#-*- coding: utf-8 -*-
#K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

inputfile = '201404.xlsx' #待聚类的数据文件
k_250 = 250                      #需要进行的聚类类别数

#读取数据并进行聚类分析
data_250 = pd.read_excel(inputfile) #读取数据

#调用k-means算法，进行聚类分析
kmodel_250 = KMeans(n_clusters = k_250, n_jobs = 2) #n_jobs是并行数，一般等于CPU数较好
kmodel_250.fit(data_250) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=250, n_init=10,
    n_jobs=2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [48]:
data_250['label']=kmodel_250.labels_
data_250.to_excel('2014_cluster_250.xlsx',index=None) #设置index=None，存为的excel文件就不会多索引列

In [49]:
center=pd.DataFrame(kmodel_250.cluster_centers_)
center.to_excel('2014center_250.xlsx',index=None)

In [50]:
# -*- coding: UTF-8 -*-
import random
import xlrd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import math

random.seed(64)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def getItems():

    fname = "2014center_250.xlsx"
    bk = xlrd.open_workbook(fname)

    try:
        sh = bk.sheet_by_name("Sheet1")
    except:
        print "no sheet in %s named Sheet1" % fname
    #获取行数
    nrows = sh.nrows
    #获取列数
    ncols = sh.ncols
    # print "nrows %d, ncols %d" % (nrows,ncols)
    #获取第一行第一列数据
    cell_value = sh.cell_value(1,1)
    row_list = []
    #获取各行数据
    items = {}
    for i in range(0,nrows):
        row_data = sh.row_values(i)
        # row_list.append(row_data)
        items[i]=row_data
    return items

items = {}
items = getItems()
toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 50)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ( 'fitness') function to be maximized

def evalStop(individual):
    cost = 0.0
    distance = 0.0
    print individual
    selected = []
    remainded = []
    temp_dist = [] #临时存储计算距离，用来比较选择剩下点到已选点最短路径
    dist = []
    cost = sum(individual)
    for index,item in enumerate(individual):
        if item == 1:
            selected.append(index)
        else:
            remainded.append(index)
    # print selected
    # print remainded
    for r in selected:
        for s in remainded:
            temp =math.sqrt((items[s][0]-items[r][0])**2+(items[s][1]-items[r][1])**2)
            temp_dist.append(temp)
        #print temp_dist
        if len(temp_dist) == 0:
            dist.append(0)
        else:
            dist.append(min(temp_dist))

    distance = sum(dist)
    return cost, distance

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalStop)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selNSGA2)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=50)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print evalStop(best_ind)
if __name__ == "__main__":
    main()


Start of evolution
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,

# k=300

In [55]:
#-*- coding: utf-8 -*-
#K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

inputfile = '201404.xlsx' #待聚类的数据文件
k_300 = 300                      #需要进行的聚类类别数

#读取数据并进行聚类分析
data_300 = pd.read_excel(inputfile) #读取数据

#调用k-means算法，进行聚类分析
kmodel_300 = KMeans(n_clusters = k_300, n_jobs = 2) #n_jobs是并行数，一般等于CPU数较好
kmodel_300.fit(data_300) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=300, n_init=10,
    n_jobs=2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [56]:
data_300['label']=kmodel_300.labels_
data_300.to_excel('2014_cluster_300.xlsx',index=None) #设置index=None，存为的excel文件就不会多索引列

In [57]:
center=pd.DataFrame(kmodel_300.cluster_centers_)
center.to_excel('2014center_300.xlsx',index=None)

In [58]:
# -*- coding: UTF-8 -*-
import random
import xlrd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import math

random.seed(64)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def getItems():

    fname = "2014center_300.xlsx"
    bk = xlrd.open_workbook(fname)

    try:
        sh = bk.sheet_by_name("Sheet1")
    except:
        print "no sheet in %s named Sheet1" % fname
    #获取行数
    nrows = sh.nrows
    #获取列数
    ncols = sh.ncols
    # print "nrows %d, ncols %d" % (nrows,ncols)
    #获取第一行第一列数据
    cell_value = sh.cell_value(1,1)
    row_list = []
    #获取各行数据
    items = {}
    for i in range(0,nrows):
        row_data = sh.row_values(i)
        # row_list.append(row_data)
        items[i]=row_data
    return items

items = {}
items = getItems()
toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 50)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ( 'fitness') function to be maximized

def evalStop(individual):
    cost = 0.0
    distance = 0.0
    print individual
    selected = []
    remainded = []
    temp_dist = [] #临时存储计算距离，用来比较选择剩下点到已选点最短路径
    dist = []
    cost = sum(individual)
    for index,item in enumerate(individual):
        if item == 1:
            selected.append(index)
        else:
            remainded.append(index)
    # print selected
    # print remainded
    for r in selected:
        for s in remainded:
            temp =math.sqrt((items[s][0]-items[r][0])**2+(items[s][1]-items[r][1])**2)
            temp_dist.append(temp)
        #print temp_dist
        if len(temp_dist) == 0:
            dist.append(0)
        else:
            dist.append(min(temp_dist))

    distance = sum(dist)
    return cost, distance

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalStop)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selNSGA2)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=50)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print evalStop(best_ind)
if __name__ == "__main__":
    main()


Start of evolution
[0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,